In [1]:
import pandas as pd
import os
import netCDF4
from netCDF4 import num2date
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np

In [2]:
def read_netcdf_file(file_location):
    f = netCDF4.Dataset(file_location) # open the nc file
    t2m = f.variables['t2m'][:].flatten()
    time = f.variables['time'][:].flatten()
    start_time = datetime.strptime("01/01/1900 00:00", "%d/%m/%Y %H:%M")
    time_points = []
    for x in range(len(time)):
        hours_from_start_time = start_time + timedelta(hours=int(time.data[x]))
        time_points.append(hours_from_start_time)
        
    # precip 'total_precipitation'
    tot_prec = f.variables['tp'][:].flatten()
    
    # wind '10m_v_component_of_wind'
    windv = f.variables['v10'][:].flatten() #'v10'][:].flatten()
    
    # wind '10m_u_component_of_wind'
    windu = f.variables['u10'][:].flatten()
    
    # surface pressure 'surface_pressure'
    surface_pressure = f.variables['sp'][:].flatten()
    
    values = pd.DataFrame({
        "t2m" : [x-273.15 for x in t2m.data if x!= -32767], # FOR '2m_temperature' - values are in Kelvin, to convert in Celsius we have to subtract 273.15
        "time" : time_points, # for the time column
        "precip" : [x for x in tot_prec.data], # FOR 'total_precipitation'
        "windv" : [x for x in windv.data if x!= -32767],  # FOR '10m_v_component_of_wind'
        "windu" : [x for x in windu.data if x!= -32767],  # FOR '10m_u_component_of_wind'
        "sfprs" : [x for x in surface_pressure.data]  # FOR 'surface_pressure'
        })
    
    return(values)

In [3]:
# https://github.com/blaylockbk/Ute_WRF/blob/master/functions/wind_calcs.py

def wind_uv_to_spd(U,V):
    """
    Calculates the wind speed from the u and v wind components
    Inputs:
      U = west/east direction (wind from the west is positive, from the east is negative)
      V = south/noth direction (wind from the south is positive, from the north is negative)
    """
    WSPD = np.sqrt(np.square(U)+np.square(V))
    return WSPD

def celcius_to_farenheit(series):
    return (series * (9/5)) + 32


def meters_second_to_mph(series):
    return series * 2.267


def meters_to_inches(series):
    return series * 39.37

# Louisiana

In [4]:
years = ['2013_2014', '2015_2016', '2017_2018', '2019_2020']#, '2021']
# locations = ['LCLA', 'NOLA', 'SCHLA']
folder = r"../raw_data/raw_weather/"

## Lake Charles

In [5]:
lcla_weather = pd.DataFrame()
for year in years:
    lcla_weather = pd.concat([lcla_weather, read_netcdf_file(f'{folder}LA/{year}_LCLA.nc')])

# Convert wind u and v components to wind speed
lcla_weather['wind_speed'] = wind_uv_to_spd(lcla_weather['windu'], lcla_weather["windv"])

# Rename all columns with location prefix
lcla_weather.rename({col:f"lcla_{col}" for col in lcla_weather.columns if col != 'time'}, axis=1, inplace=True)
lcla_weather

,lcla_t2m,time,lcla_precip,lcla_windv,lcla_windu,lcla_sfprs,lcla_wind_speed
0,16.532405,2013-01-01 00:00:00,8.497471e-07,3.476093,-2.802832,101563.991298,4.465321
1,16.486655,2013-01-01 01:00:00,8.497471e-07,3.634864,-2.907799,101511.710606,4.654840
2,16.703161,2013-01-01 02:00:00,7.860160e-06,4.145179,-2.915583,101462.112574,5.067853
3,17.292753,2013-01-01 03:00:00,8.285034e-05,4.649030,-2.422354,101426.489334,5.242259
4,17.411316,2013-01-01 04:00:00,3.796245e-04,4.758624,-1.608324,101440.526512,5.023068
...,...,...,...,...,...,...,...
17539,21.078070,2020-12-31 19:00:00,3.906191e-03,5.042856,-3.903208,100441.417697,6.376946
17540,21.345088,2020-12-31 20:00:00,2.190986e-03,5.533978,-3.463504,100359.834987,6.528459
17541,19.522163,2020-12-31 21:00:00,1.063147e-03,5.283564,-3.540271,100320.619263,6.359998
17542,18.737909,2020-12-31 22:00:00,6.667771e-04,4.771477,-2.987123,100134.257042,5.629378


## New Orleans

In [6]:
nola_weather = pd.DataFrame()
for year in years:
    nola_weather = pd.concat([nola_weather, read_netcdf_file(f'{folder}/LA/{year}_nola.nc')])
    
# Convert wind u and v components to wind speed
nola_weather['wind_speed'] = wind_uv_to_spd(nola_weather['windu'], nola_weather["windv"])
    
nola_weather.rename({col:f"nola_{col}" for col in nola_weather.columns if col != 'time'}, axis=1, inplace=True)
nola_weather

,nola_t2m,time,nola_precip,nola_windv,nola_windu,nola_sfprs,nola_wind_speed
0,15.692824,2013-01-01 00:00:00,0.000026,1.033954,-3.198926,102068.848417,3.361872
1,15.428775,2013-01-01 01:00:00,0.000020,1.618472,-3.443924,102022.188761,3.805268
2,15.393766,2013-01-01 02:00:00,0.000003,2.292124,-3.636180,101982.056621,4.298329
3,15.538548,2013-01-01 03:00:00,0.000002,2.887232,-3.270951,101943.495921,4.362938
4,15.803784,2013-01-01 04:00:00,0.000000,3.627661,-3.060264,101925.666130,4.746066
...,...,...,...,...,...,...,...
17539,21.615402,2020-12-31 19:00:00,0.001424,3.315274,-4.436793,100992.063857,5.538608
17540,20.736300,2020-12-31 20:00:00,0.002636,3.739007,-3.980970,100945.887251,5.461529
17541,20.278608,2020-12-31 21:00:00,0.001201,3.709595,-4.072350,100921.702768,5.508641
17542,19.957238,2020-12-31 22:00:00,0.001578,3.371019,-5.141312,100798.793524,6.147915


## Shreveport

In [7]:
shla_weather = pd.DataFrame()
for year in years:
    shla_weather = pd.concat([shla_weather, read_netcdf_file(f'{folder}/LA/{year}_shla.nc')])

# Convert wind u and v components to wind speed
shla_weather['wind_speed'] = wind_uv_to_spd(shla_weather['windu'], shla_weather["windv"])
    
shla_weather.rename({col:f"shla_{col}" for col in shla_weather.columns if col != 'time'}, axis=1, inplace=True)
shla_weather

,shla_t2m,time,shla_precip,shla_windv,shla_windu,shla_sfprs,shla_wind_speed
0,11.131098,2013-01-01 00:00:00,0.001336,0.519437,1.516203,100836.216792,1.602712
1,10.780504,2013-01-01 01:00:00,0.001796,-0.885304,1.184960,100848.654053,1.479153
2,10.404022,2013-01-01 02:00:00,0.001981,-1.333762,0.489028,100833.783415,1.420588
3,10.330057,2013-01-01 03:00:00,0.002427,-0.886419,-0.106351,100823.036000,0.892776
4,10.100766,2013-01-01 04:00:00,0.002835,-1.039436,-0.013303,100862.781159,1.039521
...,...,...,...,...,...,...,...
17539,6.750047,2020-12-31 19:00:00,0.001263,-2.460252,-4.218112,100139.956666,4.883166
17540,7.347660,2020-12-31 20:00:00,0.003514,-2.383978,-4.400166,99982.232340,5.004479
17541,7.186088,2020-12-31 21:00:00,0.003756,-2.203876,-4.641401,99911.571317,5.138061
17542,7.928373,2020-12-31 22:00:00,0.002979,-2.713941,-3.748115,99903.501395,4.627510


### Unit Conversions

In [8]:
# lcla_weather, nola_weather, shla_weather

lcla_weather['lcla_wind_speed'] = wind_uv_to_spd(lcla_weather['lcla_windu'],lcla_weather["lcla_windv"])
lcla_weather['lcla_wind_speed_mph'] = meters_second_to_mph(lcla_weather['lcla_wind_speed'])

nola_weather['nola_wind_speed'] = wind_uv_to_spd(nola_weather['nola_windu'],nola_weather["nola_windv"])
nola_weather['nola_wind_speed_mph'] = meters_second_to_mph(nola_weather['nola_wind_speed'])

shla_weather['shla_wind_speed'] = wind_uv_to_spd(shla_weather['shla_windu'],shla_weather["shla_windv"])
shla_weather['shla_wind_speed_mph'] = meters_second_to_mph(shla_weather['shla_wind_speed'])

# From celcius to farenheit
lcla_weather['lcla_temp_f'] = celcius_to_farenheit(lcla_weather['lcla_t2m'])
nola_weather['nola_temp_f'] = celcius_to_farenheit(nola_weather['nola_t2m'])
shla_weather['shla_temp_f'] = celcius_to_farenheit(shla_weather['shla_t2m'])

# From meters to inches
lcla_weather['lcla_precip_in'] = meters_to_inches(lcla_weather['lcla_precip'])
nola_weather['nola_precip_in'] = meters_to_inches(nola_weather['nola_precip'])
shla_weather['shla_precip_in'] = meters_to_inches(shla_weather['shla_precip'])

In [9]:
la_weather = lcla_weather.merge(nola_weather, on='time').merge(shla_weather, on='time').set_index('time').reset_index()
la_weather

,time,lcla_t2m,lcla_precip,lcla_windv,lcla_windu,lcla_sfprs,lcla_wind_speed,lcla_wind_speed_mph,lcla_temp_f,lcla_precip_in,...,nola_precip_in,shla_t2m,shla_precip,shla_windv,shla_windu,shla_sfprs,shla_wind_speed,shla_wind_speed_mph,shla_temp_f,shla_precip_in
0,2013-01-01 00:00:00,16.532405,8.497471e-07,3.476093,-2.802832,101563.991298,4.465321,10.122882,61.758329,0.000033,...,0.001011,11.131098,0.001336,0.519437,1.516203,100836.216792,1.602712,3.633349,52.035976,0.052608
1,2013-01-01 01:00:00,16.486655,8.497471e-07,3.634864,-2.907799,101511.710606,4.654840,10.552522,61.675979,0.000033,...,0.000798,10.780504,0.001796,-0.885304,1.184960,100848.654053,1.479153,3.353240,51.404907,0.070710
2,2013-01-01 02:00:00,16.703161,7.860160e-06,4.145179,-2.915583,101462.112574,5.067853,11.488823,62.065690,0.000309,...,0.000120,10.404022,0.001981,-1.333762,0.489028,100833.783415,1.420588,3.220474,50.727240,0.077989
3,2013-01-01 03:00:00,17.292753,8.285034e-05,4.649030,-2.422354,101426.489334,5.242259,11.884201,63.126956,0.003262,...,0.000080,10.330057,0.002427,-0.886419,-0.106351,100823.036000,0.892776,2.023924,50.594103,0.095547
4,2013-01-01 04:00:00,17.411316,3.796245e-04,4.758624,-1.608324,101440.526512,5.023068,11.387295,63.340369,0.014946,...,0.000000,10.100766,0.002835,-1.039436,-0.013303,100862.781159,1.039521,2.356594,50.181379,0.111613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70123,2020-12-31 19:00:00,21.078070,3.906191e-03,5.042856,-3.903208,100441.417697,6.376946,14.456536,69.940526,0.153787,...,0.056053,6.750047,0.001263,-2.460252,-4.218112,100139.956666,4.883166,11.070137,44.150084,0.049725
70124,2020-12-31 20:00:00,21.345088,2.190986e-03,5.533978,-3.463504,100359.834987,6.528459,14.800017,70.421159,0.086259,...,0.103770,7.347660,0.003514,-2.383978,-4.400166,99982.232340,5.004479,11.345154,45.225789,0.138343
70125,2020-12-31 21:00:00,19.522163,1.063147e-03,5.283564,-3.540271,100320.619263,6.359998,14.418115,67.139893,0.041856,...,0.047279,7.186088,0.003756,-2.203876,-4.641401,99911.571317,5.138061,11.647984,44.934959,0.147884
70126,2020-12-31 22:00:00,18.737909,6.667771e-04,4.771477,-2.987123,100134.257042,5.629378,12.761800,65.728236,0.026251,...,0.062137,7.928373,0.002979,-2.713941,-3.748115,99903.501395,4.627510,10.490564,46.271071,0.117283


In [10]:
la_weather.to_csv(r"../data/weather/la_weather_full.csv", index=False)

# Michigan

In [11]:
# det_weather
det_weather = pd.read_csv(r'../raw_data/raw_weather/MI/det_weather.csv')
det_weather.rename({col: f"det_{col}" for col in det_weather.columns if col != 'time'}, axis=1, inplace=True)


# lan_weather
lan_weather = pd.read_csv(r'../raw_data/raw_weather/MI/lan_weather.csv')
lan_weather.rename({col: f"lan_{col}" for col in lan_weather.columns if col != 'time'}, axis=1, inplace=True)

# gr_weather
gr_weather = pd.read_csv(r'../raw_data/raw_weather/MI/gr_weather.csv')
gr_weather.rename({col: f"gr_{col}" for col in gr_weather.columns if col != 'time'}, axis=1, inplace=True)


### Unit conversions

In [12]:
det_weather['det_wind_speed'] = wind_uv_to_spd(det_weather['det_windu'],det_weather["det_windv"])
det_weather['det_wind_speed_mph'] = meters_second_to_mph(det_weather['det_wind_speed'])
det_weather['det_precip_in'] = meters_to_inches(det_weather['det_precip'])
det_weather['det_temp_f'] = celcius_to_farenheit(det_weather['det_t2m'])

lan_weather['lan_wind_speed'] = wind_uv_to_spd(lan_weather['lan_windu'],lan_weather["lan_windv"])
lan_weather['lan_wind_speed_mph'] = meters_second_to_mph(lan_weather['lan_wind_speed'])
lan_weather['lan_precip_in'] = meters_to_inches(lan_weather['lan_precip'])
lan_weather['lan_temp_f'] = celcius_to_farenheit(lan_weather['lan_t2m'])

gr_weather['gr_wind_speed'] = wind_uv_to_spd(gr_weather['gr_windu'],gr_weather["gr_windv"])
gr_weather['gr_wind_speed_mph'] = meters_second_to_mph(gr_weather['gr_wind_speed'])
gr_weather['gr_precip_in'] = meters_to_inches(gr_weather['gr_precip'])
gr_weather['gr_temp_f'] = celcius_to_farenheit(gr_weather['gr_t2m'])

### Put all Michigan weather data together

In [13]:
michigan_weather = det_weather.merge(lan_weather, on='time')
michigan_weather = michigan_weather.merge(gr_weather, on='time') # Not done yet
michigan_weather['time'] = pd.to_datetime(michigan_weather['time'])
michigan_weather.set_index('time', inplace=True)#.reset_index()
michigan_weather.reset_index(inplace=True)
michigan_weather

,time,det_t2m,det_precip,det_windv,det_windu,det_sfprs,det_wind_speed,det_wind_speed_mph,det_precip_in,det_temp_f,...,lan_temp_f,gr_t2m,gr_precip,gr_windv,gr_windu,gr_sfprs,gr_wind_speed,gr_wind_speed_mph,gr_precip_in,gr_temp_f
0,2013-12-19 00:00:00,-5.199975,0.000000e+00,5.849919,1.730580,99328.723333,6.100529,13.829899,0.000000e+00,22.640045,...,23.630686,-3.924434,0.000000e+00,5.843718,0.944893,98365.813627,5.919616,13.419770,0.000000,24.936019
1,2013-12-19 01:00:00,-5.596619,0.000000e+00,6.476971,1.300262,99222.710027,6.606196,14.976247,0.000000e+00,21.926086,...,23.942175,-3.452017,0.000000e+00,6.344644,2.551897,98323.293361,6.838617,15.503146,0.000000,25.786370
2,2013-12-19 02:00:00,-5.503398,0.000000e+00,6.557646,2.529697,99151.216485,7.028662,15.933977,0.000000e+00,22.093883,...,25.588847,-2.492659,0.000000e+00,5.958250,3.470401,98276.604441,6.895247,15.631525,0.000000,27.513214
3,2013-12-19 03:00:00,-4.168153,0.000000e+00,6.244639,3.580197,99086.512376,7.198148,16.318201,0.000000e+00,24.497325,...,27.813071,-1.325711,0.000000e+00,5.338338,3.963006,98241.837870,6.648554,15.072273,0.000000,29.613721
4,2013-12-19 04:00:00,-3.712104,0.000000e+00,5.863422,4.345900,99034.160126,7.298395,16.545461,0.000000e+00,25.318213,...,28.132672,-0.906258,0.000000e+00,4.714963,4.111512,98212.824041,6.255830,14.181967,0.000000,30.368735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61675,2020-12-31 19:00:00,0.431772,-8.673617e-19,-0.309489,1.489445,100210.267826,1.521259,3.448695,-3.414803e-17,32.777190,...,31.005366,-1.528080,3.671919e-07,0.575787,1.285144,99678.413478,1.408235,3.192468,0.000014,29.249457
61676,2020-12-31 20:00:00,0.570731,-8.673617e-19,-0.050178,0.958026,100242.405517,0.959339,2.174822,-3.414803e-17,33.027316,...,31.002077,-1.525492,0.000000e+00,0.709980,1.026244,99698.888962,1.247897,2.828983,0.000000,29.254114
61677,2020-12-31 21:00:00,0.500338,-8.673617e-19,0.328743,0.248226,100282.005258,0.411932,0.933850,-3.414803e-17,32.900608,...,31.284944,-1.466840,0.000000e+00,0.781532,1.170450,99764.289561,1.407389,3.190552,0.000000,29.359688
61678,2020-12-31 22:00:00,-2.045718,-8.673617e-19,1.292628,0.151420,100337.037874,1.301467,2.950426,-3.414803e-17,28.317707,...,29.072985,-2.009373,9.179797e-07,0.758615,1.147980,99860.619373,1.375993,3.119375,0.000036,28.383129


In [14]:
michigan_weather.to_csv(r'../data/weather/mi_weather_full.csv', index=False)

In [15]:
weather_full = michigan_weather.merge(la_weather, on='time')
weather_full.to_csv(r'../data/weather/weather_full.csv', index=False)